In [1]:
def get_vacancy_skills(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'

    response = requests.get(url)
    response.raise_for_status()
    data = response.json()

    skills = [skill['name'] for skill in data.get('key_skills', [])]
    if not skills:
            return "Не указано"
    else:
        return ', '.join(skills)

In [2]:
import requests
import pandas as pd
from datetime import datetime

def get_vacancy_skills(vacancy_id):
    """
    Получает список навыков для указанной вакансии.

    Args:
        vacancy_id: ID вакансии.

    Returns:
        Список навыков (строк).  Возвращает пустой список в случае ошибки.
    """

    url = f"https://api.hh.ru/vacancies/{vacancy_id}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        skills = [skill["name"] for skill in data.get("key_skills", [])]
        return skills
    else:
        print(f"Ошибка при получении навыков для вакансии {vacancy_id}: {response.status_code}, {response.text}")
        return []

def search_vacancies(text="", area=1, max_vacancies=1):
    """
    Ищет вакансии с указанием текста и региона.

    Args:
        text: Текст для поиска в вакансиях.
        area: ID региона.

    Returns:
        Кортеж: (vacancies_data, skills_list)
            vacancies_Список словарей с информацией о вакансиях.
            skills_list: Список словарей с навыками для каждой вакансии.
    """
    url = "https://api.hh.ru/vacancies"
    params = {
        "text": text,
        "area": area,
        "per_page": 100  # Максимальное количество вакансий на странице (можно менять)
    }
    vacancies_data = []  # Список для хранения информации о вакансиях
    skills_list = [] # Список для хранения скиллов
    total_vacancies = 0 # Счетчик извлеченных вакансий
    page = 0
    while True: #Перебираем все страницы
        params["page"] = page
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            items = data.get("items", [])  # Получаем список вакансий с текущей страницы

            if not items:  # Если вакансий на странице нет, выходим из цикла
                break

            for item in items:
                published_at = item["published_at"]
                try:
                   published_at_formatted = datetime.fromisoformat(published_at.replace('Z', '+00:00')).strftime('%Y-%m-%d %H:%M')
                except ValueError:
                    published_at_formatted = 'Не указано'

                vacancy_info = {
                    "id": item["id"],
                    "name": item["name"],
                    "employer": item["employer"]["name"],
                    "salary_from": item["salary"]["from"] if item["salary"] and item["salary"].get("from") else 'Не указано',
                    "experience": item["experience"]["name"] if "experience" in item else 'Не указано',
                    "description": item.get("snippet", {}).get("requirement", 'Не указано'), # Используем snippet для краткого описания
                    "published_at": published_at_formatted,
                }

                skills_for_vacancy = get_vacancy_skills(item['id']) # Получаем список навыков для вакансии
                for skill in skills_for_vacancy:
                  skills_list.append({"vacancy_id": item["id"], "skill": skill}) # Записываем в skills_list каждый скил отдельной строкой.

                vacancies_data.append(vacancy_info)
                total_vacancies += 1 # Инкрементируем счетчик

            if total_vacancies >= max_vacancies: # Проверяем лимит и выходим из цикла pages
                break

            page += 1

            if page >= data.get("pages", 1):  # Если текущая страница последняя, выходим из цикла
                break

        else:
            print(f"Ошибка при запросе: {response.status_code}, {response.text}")
            return [], [] # Возвращаем пустые списки в случае ошибки

    return vacancies_data, skills_list


# Пример использования:
if __name__ == '__main__':
    text = ['data engineer', 'gamedev', 'java', '.net', 'реклама', 'маркетинг']
    df_vacancy = pd.DataFrame()
    df_vacancy_skills = pd.DataFrame()

    for i in text:
        vacancies, skills = search_vacancies(
            text=i,
            area=[1,2,22],
            max_vacancies=1,
        )

        if vacancies:
            df_vacancy = pd.concat([df_vacancy, pd.DataFrame(vacancies)], ignore_index=True)
        if skills:
            df_vacancy_skills = pd.concat([df_vacancy_skills, pd.DataFrame(skills)], ignore_index=True)

    print("DataFrame с вакансиями:")
    print(df_vacancy.head())

    print("\nDataFrame с навыками:")
    print(df_vacancy_skills.head())

Ошибка при получении навыков для вакансии 118469860: 403, {"errors":[{"value":"captcha_required","captcha_url":"https://hh.ru/account/captcha?state=pxvcxBozfu7ry7R4QCetFmUA3SFZr_Pa4yXPQEusAiTHBc8_LWa626j-S227_PWg9_aifZ2ny8O96aapoJBqYAKdo0Xt38LgKJ5fyfbA5Pw%3D","type":"captcha_required"}],"request_id":"17424389485745248c34705d7e44148f"}
Ошибка при получении навыков для вакансии 118549505: 403, {"errors":[{"value":"captcha_required","captcha_url":"https://hh.ru/account/captcha?state=pxvcxBozfu7ry7R4QCetFmUA3SFZr_Pa4yXPQEusAiTHBc8_LWa626j-S227_PWg9_aifZ2ny8O96aapoJBqYPJ9HmskBRDxW5ndIh1MdBc%3D","type":"captcha_required"}],"request_id":"17424389489340df3a267fb6ad9c0f7a"}
Ошибка при получении навыков для вакансии 118432324: 403, {"errors":[{"value":"captcha_required","captcha_url":"https://hh.ru/account/captcha?state=pxvcxBozfu7ry7R4QCetFmUA3SFZr_Pa4yXPQEusAiTHBc8_LWa626j-S227_PWg9_aifZ2ny8O96aapoJBqYIdP_2u1LP4Az1-FVTzC7cA%3D","type":"captcha_required"}],"request_id":"17424389493066fae86d4425

In [3]:
df_vacancy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            600 non-null    object
 1   name          600 non-null    object
 2   employer      600 non-null    object
 3   salary_from   600 non-null    object
 4   experience    600 non-null    object
 5   description   597 non-null    object
 6   published_at  600 non-null    object
dtypes: object(7)
memory usage: 32.9+ KB


In [4]:
df_vacancys = df_vacancy.copy()
df_vacancy_skillss = df_vacancy_skills.copy()

In [5]:
df_vacancy = df_vacancy.drop_duplicates(subset=['id'])

In [6]:
df_vacancy_skills

,vacancy_id,skill
0,118550274,Подбор персонала
1,118550274,Адаптация персонала
2,118550274,Прямой поиск
3,118550274,Оценка кандидатов
4,118550274,IT Recruitment
...,...,...
1956,118468826,Flask
1957,118468826,FastAPI
1958,118468826,ORM
1959,118468826,MySQL


In [7]:
unique_skills = df_vacancy_skills['skill'].unique()
data_unique_skill = {'skill': unique_skills}
df_unique_skill = pd.DataFrame(data_unique_skill)
df_unique_skill['id'] = range(1, len(df_unique_skill) + 1) # Создаем новый id для каждого уникального навыка
df_unique_skill = df_unique_skill[['id', 'skill']]  # Меняем порядок столбцов
print("df_unique_skill:\n", df_unique_skill)

df_unique_skill:
       id                skill
0      1     Подбор персонала
1      2  Адаптация персонала
2      3         Прямой поиск
3      4    Оценка кандидатов
4      5       IT Recruitment
..   ...                  ...
810  811          Google Docs
811  812     Удаленная работа
812  813              продажи
813  814         маркетплейсы
814  815            аналитика

[815 rows x 2 columns]


In [8]:
df_merged = pd.merge(df_vacancy_skills, df_unique_skill, on='skill', how='left')

# 2. Замена столбца 'skill' в df_skill на значения из столбца 'id' из df_unique_skill
#  Столбец skill будет перезаписан id соответствующего навыка
df_vacancy_skills['skill'] = df_merged['id']

In [9]:
df_vacancy_skills.insert(0, 'id', range(1, len(df_vacancy_skills) + 1))
df_vacancy_skills = df_vacancy_skills.rename(columns={'skill': 'skill_id'})
df_vacancy_skills

,id,vacancy_id,skill_id
0,1,118550274,1
1,2,118550274,2
2,3,118550274,3
3,4,118550274,4
4,5,118550274,5
...,...,...,...
1956,1957,118468826,39
1957,1958,118468826,40
1958,1959,118468826,41
1959,1960,118468826,42


In [10]:
df_unique_skill

,id,skill
0,1,Подбор персонала
1,2,Адаптация персонала
2,3,Прямой поиск
3,4,Оценка кандидатов
4,5,IT Recruitment
...,...,...
810,811,Google Docs
811,812,Удаленная работа
812,813,продажи
813,814,маркетплейсы


In [11]:
df_vacancy

,id,name,employer,salary_from,experience,description,published_at
0,118550274,Ведущий специалист по подбору персонала (офис),Гознак,Не указано,От 3 до 6 лет,Что хотим видеть у кандидата: - успешный опыт ...,2025-03-19 12:12
1,118460000,QA engineer DWH,Ренессанс Банк,Не указано,От 1 года до 3 лет,Высшее техническое образование. Опыт проведени...,2025-03-17 16:05
2,118524897,Junior Data Scientist / Machine Learning Engineer,Титов Дмитрий Владимирович,150000,Нет опыта,Навыки работы с системами контроля версий (Git...,2025-03-18 20:14
3,118567600,Стажер аналитик / разработчик (SQL / Python),КОРУС Консалтинг,Не указано,Нет опыта,Ты в том числе учишься на 3-4 курсе бакалавриа...,2025-03-19 16:32
4,118567601,Стажер аналитик / разработчик (SQL / Python),КОРУС Консалтинг,Не указано,Нет опыта,Ты учишься на 3-4 курсе бакалавриата или 1-2 к...,2025-03-19 16:32
...,...,...,...,...,...,...,...
589,117406897,Менеджер по работе с отзывами,Smirnov.Marketing,30000,Нет опыта,Вы умеете обрабатывать возражения клиентов. Вы...,2025-02-18 11:09
592,118512914,Директор по маркетингу,Concept Group,Не указано,От 3 до 6 лет,Высшее профильное образование. Аналогичный опы...,2025-03-18 15:47
593,118276879,Графический дизайнер контента для маркетплейсов,Мартюченко Ян Игоревич,80000,От 1 года до 3 лет,Внимательность к деталям и способность работат...,2025-03-13 08:52
594,118331694,Менеджер продукта/Product manager,Лит Энерджи,Не указано,От 3 до 6 лет,Опыт вывода новых продуктов на рынок. Навык по...,2025-03-14 10:29


In [21]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.dialects.postgresql import UUID
import uuid

# Параметры подключения к PostgreSQL
db_user = "sqlmaster"
db_password = "VOf67bY6kR"
db_host = "92.63.177.19"
db_name = "hh_ithub_msod"
schema_name = "jobsearch"

# Формируем строку подключения
db_string = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"

# Создаем движок SQLAlchemy
engine = create_engine(db_string)

# Создаем базовый класс для моделей
Base = declarative_base()

In [29]:
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy.dialects.postgresql import UUID
import uuid

# Параметры подключения к PostgreSQL
db_user = "sqlmaster"
db_password = "VOf67bY6kR"
db_host = "92.63.177.19"
db_name = "hh_ithub_msod"
schema_name = "jobsearch"

# Формируем строку подключения
db_string = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"

# Создаем движок SQLAlchemy
engine = create_engine(db_string)

# Создаем базовый класс для моделей
Base = declarative_base()


class Vacancy(Base):
    __tablename__ = 'vacancies'
    __table_args__ = {'schema': schema_name}

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    employer = Column(String)
    salary_from = Column(String)
    experience = Column(String)
    description = Column(String)
    published_at = Column(DateTime)

    vacancy_skills = relationship("VacancySkill", back_populates="vacancy")

class UniqueSkill(Base):
    __tablename__ = 'skills'
    __table_args__ = {'schema': schema_name}

    id = Column(Integer, primary_key=True, autoincrement=True)
    skill = Column(String(255), nullable=False, unique=True)

    vacancy_skills = relationship("VacancySkill", back_populates="skill")
    student_skills = relationship("StudentSkill", back_populates="skill")

class VacancySkill(Base):
    __tablename__ = 'vacancy_skills'
    __table_args__ = {'schema': schema_name}

    id = Column(Integer, primary_key=True, autoincrement=True)
    vacancy_id = Column(Integer, ForeignKey(f'{schema_name}.vacancies.id', ondelete='CASCADE'), nullable=False)
    skill_id = Column(Integer, ForeignKey(f'{schema_name}.skills.id', ondelete='CASCADE'), nullable=False)

    vacancy = relationship("Vacancy", back_populates="vacancy_skills")
    skill = relationship("UniqueSkill", back_populates="vacancy_skills")

class Students(Base):
    __tablename__ = 'students'
    __table_args__ = {'schema': schema_name}

    id = Column(Integer, primary_key=True, autoincrement=True)
    full_name = Column(String(255), nullable=False)

    student_skills = relationship("StudentSkill", back_populates="student")

class StudentSkill(Base):
    __tablename__ = 'student_skills'
    __table_args__ = {'schema': schema_name}

    id = Column(Integer, primary_key=True, autoincrement=True)
    student_id = Column(Integer, ForeignKey(f'{schema_name}.students.id', ondelete='CASCADE'), nullable=False)
    skill_id = Column(Integer, ForeignKey(f'{schema_name}.skills.id', ondelete='CASCADE'), nullable=False)

    student = relationship("Students", back_populates="student_skills")
    skill = relationship("UniqueSkill", back_populates="student_skills")

Base.metadata.create_all(engine)

In [30]:
vacancy_data = df_vacancy.to_dict(orient='records')

skill_data = df_vacancy_skills.to_dict(orient='records')

skill_unique_data = df_unique_skill.to_dict(orient='records')


# Создание сессии SQLAlchemy
Session = sessionmaker(bind=engine)
session = Session()

# Добавление данных в таблицу vacancies
vacancies = [Vacancy(**vacancy) for vacancy in vacancy_data]
session.add_all(vacancies)

# Добавление данных в таблицу skills
skills = [UniqueSkill(**skill) for skill in skill_unique_data]
session.add_all(skills)

skills_vacancy = [VacancySkill(**skill) for skill in skill_data]
session.add_all(skills_vacancy)

# Фиксация изменений и закрытие сессии
session.commit()
session.close()

print("Данные успешно загружены в базу данных!")

Данные успешно загружены в базу данных!


In [31]:
# Сохранение данных из df_vacancy в CSV
df_vacancy.to_csv('vacancies.csv', index=False, encoding='utf-8')

# Сохранение данных из df_vacancy_skills в CSV
df_vacancy_skills.to_csv('vacancy_skills.csv', index=False, encoding='utf-8')

# Сохранение данных из df_unique_skill в CSV
df_unique_skill.to_csv('unique_skills.csv', index=False, encoding='utf-8')

print("Данные успешно сохранены в CSV файлы!")

Данные успешно сохранены в CSV файлы!


In [33]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

# Параметры подключения к базе данных
db_user = "sqlmaster"
db_password = "VOf67bY6kR"
db_host = "92.63.177.19"
db_name = "hh_ithub_msod"
schema_name = "jobsearch"

# Создаем движок для подключения к базе данных
db_string = f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}"
engine = create_engine(db_string)

# Создаем класс "Session" для сессий с базой данных
Session = sessionmaker(bind=engine)
# Создаем сессию
session = Session()

# Запрашиваем и экспортируем данные из таблицы Vacancies
vacancies_query = session.execute(text(f"SELECT * FROM {schema_name}.vacancies"))
vacancies_df = pd.DataFrame(vacancies_query.fetchall(), columns=vacancies_query.keys())
vacancies_df.to_csv('vacancies_export.csv', index=False)

# Запрашиваем и экспортируем данные из таблицы Skills
skills_query = session.execute(text(f"SELECT * FROM {schema_name}.skills"))
skills_df = pd.DataFrame(skills_query.fetchall(), columns=skills_query.keys())
skills_df.to_csv('skills_export.csv', index=False)

# Запрашиваем и экспортируем данные из таблицы Vacancy Skills
vacancy_skills_query = session.execute(text(f"SELECT * FROM {schema_name}.vacancy_skills"))
vacancy_skills_df = pd.DataFrame(vacancy_skills_query.fetchall(), columns=vacancy_skills_query.keys())
vacancy_skills_df.to_csv('vacancy_skills_export.csv', index=False)

# Запрашиваем и экспортируем данные из таблицы Students
students_query = session.execute(text(f"SELECT * FROM {schema_name}.students"))
students_df = pd.DataFrame(students_query.fetchall(), columns=students_query.keys())
students_df.to_csv('students_export.csv', index=False)

# Запрашиваем и экспортируем данные из таблицы Student Skills
student_skills_query = session.execute(text(f"SELECT * FROM {schema_name}.student_skills"))
student_skills_df = pd.DataFrame(student_skills_query.fetchall(), columns=student_skills_query.keys())
student_skills_df.to_csv('student_skills_export.csv', index=False)

# Закрываем сессию
session.close()

print("Данные успешно экспортированы в файлы CSV.")

Данные успешно экспортированы в файлы CSV.
